# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [136]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, f1_score

In [36]:
data = pd.read_csv('data/labeled.csv')

In [150]:
class Experiment:

    train, test = train_test_split(data, test_size=0.1, shuffle=True)
    train.reset_index(inplace=True)
    test.reset_index(inplace=True)

    def __init__(
            self,
            vectorizer_model: object,
            vectorizer_args : dict,
            classifier_model: object,
            classifier_args : dict
        ):
        
        self.vectorizer = vectorizer_model(**vectorizer_args)
        X = self.vectorizer.fit_transform(self.train.comment)
        y = self.train.toxic.values

        self.classifier = classifier_model(**classifier_args)
        self.classifier.fit(X, y)

    def get_test_preds(self):
        X = self.vectorizer.transform(self.test.comment)
        preds = self.classifier.predict(X)
        return preds

    def get_report(self):
        return classification_report(
            self.test.toxic.values,
            self.get_test_preds(),
            zero_division=0
        )
    
    def get_f1(self):
        return f1_score(
            self.test.toxic.values,
            self.get_test_preds(),
            zero_division=0        
        )

In [151]:
experiment_default = Experiment(
    TfidfVectorizer,
    {},
    MultinomialNB,
    {}
)

In [152]:
print(
    experiment_default.get_report()
)

              precision    recall  f1-score   support

         0.0       0.74      0.99      0.85       977
         1.0       0.94      0.25      0.40       465

    accuracy                           0.75      1442
   macro avg       0.84      0.62      0.62      1442
weighted avg       0.80      0.75      0.70      1442



In [153]:
import razdel

In [154]:
## Если просто сунуть по ссылке, прилетает TypeError: '<' not supported between instances of 'Substring' and 'Substring'

def odel_tokenize(text):
    return [_.text for _ in razdel.tokenize(text)]

In [155]:
experiment_razdel = Experiment(
    TfidfVectorizer,
    {
        "tokenizer": odel_tokenize,
        "analyzer" : "word"
    },
    MultinomialNB,
    {}
)

d:\HSE subj data\актуальные проблемы компьютерной лингвистики\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [156]:
print(
    experiment_razdel.get_report()
)

              precision    recall  f1-score   support

         0.0       0.72      0.99      0.83       977
         1.0       0.93      0.17      0.29       465

    accuracy                           0.73      1442
   macro avg       0.82      0.58      0.56      1442
weighted avg       0.79      0.73      0.66      1442



#### Комментарий

В документации Razdel написано, что

<q>
Правила в Razdel оптимизированы для аккуратно написанных текстов с правильной пунктуацией. Решение хорошо работает с новостными статьями, художественными текстами. На постах из социальных сетей, расшифровках телефонных разговоров качество ниже.
</q>

Поэтому неудивительно, что на данном датасете Razdel работает чуть хуже.

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [233]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [235]:
experiment_params = (

    {
        "classifier_model": MultinomialNB,
        "classifier_args" : {},  ##  Чудесным образом любое вмешательство в парамеры сильно ухудшает результаты
        "vectorizer_model": CountVectorizer,
        ##  Здесь и далее наилучшие результаты давала векторизация с параметрами analyzer + ngram_range + max_df
        ##  Т.е. с 3-мя вручную заданными параметрами, поэтому 2 отставшихся по заданию забил дефолтными.
        ##  Иные вмешательства делали результат хуже.
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

    ##  Фактически задание не выполнено т.к. в обоих случаях результаты получаются лучше с мешком слов.
    ##  Однако вот такая конфигурация проходит только с варнингом, поэтому можно считать, что TF-IDF лучше т.к. стабильнее (?)
    {   
        "classifier_model": LogisticRegression,
        "classifier_args" : {"solver":"liblinear", "penalty":"l2"},
        "vectorizer_model": CountVectorizer,
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

    {
        "classifier_model": LogisticRegression,
        "classifier_args" : {"solver":"liblinear", "penalty":"l2"},
        "vectorizer_model": TfidfVectorizer,
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

)

In [236]:
finished_experiments = []

for d in experiment_params:
    experiment = Experiment(**d)
    print(f"{d["classifier_model"].__name__}\t{d["vectorizer_model"].__name__}")
    print("F1: ", experiment.get_f1())
    print(experiment.get_report())
    finished_experiments.append(experiment)

MultinomialNB	CountVectorizer
F1:  0.8549107142857143
              precision    recall  f1-score   support

         0.0       0.92      0.95      0.93       977
         1.0       0.89      0.82      0.85       465

    accuracy                           0.91      1442
   macro avg       0.90      0.89      0.89      1442
weighted avg       0.91      0.91      0.91      1442



d:\HSE subj data\актуальные проблемы компьютерной лингвистики\.venv\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LogisticRegression	CountVectorizer
F1:  0.843400447427293
              precision    recall  f1-score   support

         0.0       0.91      0.95      0.93       977
         1.0       0.88      0.81      0.84       465

    accuracy                           0.90      1442
   macro avg       0.90      0.88      0.89      1442
weighted avg       0.90      0.90      0.90      1442

LogisticRegression	TfidfVectorizer
F1:  0.8133971291866029
              precision    recall  f1-score   support

         0.0       0.88      0.97      0.92       977
         1.0       0.92      0.73      0.81       465

    accuracy                           0.89      1442
   macro avg       0.90      0.85      0.87      1442
weighted avg       0.89      0.89      0.89      1442



In [239]:
##  Для читаемости
nb_bow_preds       = finished_experiments[0].get_test_preds()
logreg_bow_preds   = finished_experiments[1].get_test_preds()
logreg_tfidf_preds = finished_experiments[2].get_test_preds()

In [252]:
same_normal_results      = []
same_toxic_results       = []
different_normal_results = []
different_toxic_results  = []

# for comment, is_toxic, nb_bow_pred, logreg_bow_pred, logreg_tfidf_pred \
for comment, results in zip(
        Experiment.test.comment,
        zip(
            experiment.test.toxic,
            nb_bow_preds,
            logreg_bow_preds,
            logreg_tfidf_preds
        )
    ):

    elem_to_append = comment.strip() + ": " + " ".join(map(str, results))
    if sum(results) in (0, 4):
        if results[0] == 1:
            same_toxic_results.append(elem_to_append)
        else:
            same_normal_results.append(elem_to_append)
    else:
        if results[0] == 1:
            different_toxic_results.append(elem_to_append)
        else:
            different_normal_results.append(elem_to_append)

In [258]:
len(same_toxic_results), len(same_normal_results), len(different_toxic_results), len(different_normal_results)

(323, 905, 142, 72)

In [253]:
for c in same_toxic_results[:10]:
    print(c)

Нахуй иди, я тебе весь тред что ли читать буду? Пидор, бешбармак тебе в хычин!: 1.0 1.0 1.0 1.0
Короче типичное гиперкомпенсаторство от малых народов-неудачников И дрочь на нацистскую германию это просто эталонный признак даунов-аутистов, для которых эстетика важнее мозга и своей культуры.: 1.0 1.0 1.0 1.0
Павел Михайловский казалось бы, причём здесь хохлы: 1.0 1.0 1.0 1.0
А я бы сдал Путина, согласятся ли хохлы получить Путина и отказаться от Крыма?: 1.0 1.0 1.0 1.0
Красавчик, надеюсь свалит за бугор и продолжит шатать пидорашкку изнутри и ставить Ванек на путь истинный.: 1.0 1.0 1.0 1.0
И зачем в сноске эти откровения Не виляй жопой, челядь. Твоя претензия была к тому, что ссылка якобы взята из непроверенного источника, во что я тебя и ткнул твоей неумытой харей. Ты пойман за руку как подлец и дешевка.: 1.0 1.0 1.0 1.0
Ну и тебя тоже к стенке. Лично пулю в лоб всажу, ибо ну их в жопу ваши революции , наелась ими уже страна.: 1.0 1.0 1.0 1.0
Котенок не блевотина же.: 1.0 1.0 1.0 1.0
Х

In [254]:
for c in same_normal_results[:10]:
    print(c)

Там гайка была под ключ на 17 и затягивалась с такой силой, что контакт никогда не пропадал. Лично у меня не бывало таких проблем. Но теоретически, они могут возникнуть. Вот одна из схем, которые мне дал яндекс, четко видно, что сначала на стартер идет, а потом уже дальше.: 0.0 0.0 0.0 0.0
А я так фанатела от орифлеймовских духов, стоили они аж 1200 рублей, а пахли пудрой. Тоже уже не производят, увы(: 0.0 0.0 0.0 0.0
Один кондей дешевле чем каналка. Мощности только на номер а не на всех здание в холостую: 0.0 0.0 0.0 0.0
Есть ощущение, что мое рабочее здание из Окленда. Крыша течет, вода по стенам, везде плесень. А ведь ремонт 6 лет назад сделали на 5 лимонов, мля. И все сроки исправления оттянули до невзврата, суки. Все, кто сюда приходят, постоянно болеют давлением, достаточно прийти с простудой, чтобы заразились все посетители, посидевшие у нас пару часов.: 0.0 0.0 0.0 0.0
У этих людей работа такая. Ну и средств получения информации гораздо больше чем у рядового, да даже не рядовог

In [255]:
for c in different_toxic_results[:10]:
    print(c)

Скажите участковому, он её застрелит.: 1.0 0.0 0.0 0.0
До чего же ельцинская банда довела дороги в городке...: 1.0 0.0 1.0 0.0
А девушек-сумоисток жирдяй-тян? :-): 1.0 1.0 0.0 0.0
Ну а что такого то? Ясен хуй если он завел трактор то особой любви к рашке он не испытывал. Высказывает свое мнение. Все правильно говорит. Разбил розовые очки в один прекрасный день и увидел все как есть. Что он не так написал? Что люди тут друг друга не ненавидят? Все правильно пишет. Целиком читать не буду, мне времени жалко.: 1.0 1.0 1.0 0.0
А вы похоже тоже из этих? Не буду, пожалуй, вам ничего писать, вдруг вы заразны.: 1.0 0.0 0.0 0.0
Да, на Rust е ЯП были примеры задач по этому тупому примеру.: 1.0 0.0 1.0 0.0
Долбоеб, это молоко для веганов и тех у кого непереносимость лактозы. В ваших колхозах про веганов слышали вообще? Хотя кого я спрашиваю, нищий чубатый деревенщина, ничего сложнее сала не видел. И кстати эта хуйня стоит дороже обычного молока, охуеть да?: 1.0 1.0 1.0 0.0
гнильем скотину кормить?

In [257]:
for c in different_normal_results[:10]:
    print(c)

Первая заявка на победу в треде! Кемерово теперь славный город Пыня. Если появится другой претендендент - Кемерово станет Пыня-1, закрытое территориальное образование по типу Челябинска-40.: 0.0 1.0 1.0 0.0
Да не торопись ты так, бабуль: 0.0 1.0 1.0 1.0
А что, до 05.11.17 кто-то другой это смотрел?: 0.0 1.0 0.0 0.0
Мне Вас очень жаль, если для Вас оскорбления - норма.: 0.0 1.0 1.0 0.0
Краска с радиопоглащением, которой окрашен самолёт будет мешать радару с фазированной решеткой: 0.0 0.0 1.0 0.0
Горький так выступал за диктатуру пролетариата, а когда она началась ему почему-то захотелось жить на загнивающем Западе.: 0.0 1.0 1.0 0.0
Пока гром не грянет мужик не перекрестится.: 0.0 1.0 0.0 0.0
Извини тебе в грязном ходить приятно?: 0.0 1.0 1.0 1.0
Если мы не сделаем снеговик, его сделают солдаты НАТО!: 0.0 1.0 1.0 1.0
хочешь грудь модератора?: 0.0 1.0 0.0 1.0


#### Комментарий

Токсичность комментариев -- очень субъективная тема, особенно в рамках бинарной классификации. Кого-то может и вот такое задеть: ```"хочешь грудь модератора?"``` (последний пример из списка выше). Конечно, для такой задачи нужно бы иметь побольше классов.

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [6]:
# ваш код